# Chroma
`pip install langchain-chroma`
Chroma是一个 AI 原生开源矢量数据库，专注于提高开发人员的工作效率和幸福感。Chroma 已获得 Apache 2.0 许可。

Chroma 以多种模式运行。请参阅下文中与 LangChain 集成的示例。
- `in-memory`- 在 Python 脚本或 Jupyter Notebook 中
- `in-memory with persistance`- 在脚本或笔记本中保存/加载到磁盘
- `in a docker container`- 作为在本地机器或云端运行的服务器

与任何其他数据库一样，您可以：
- .add
- .get
- .update
- .upsert
- .delete
- .peek
- 并.query运行相似性搜索。       
查看完整文档：[docs](https://docs.trychroma.com/)。要直接访问这些方法，您可以这样做._collection.method()

<img src="https://docs.trychroma.com/img/hrm4.svg">

## chromadb 简单尝试


In [4]:
###  pip install chromadb
## 创建 Chroma 客户端
import chromadb
chroma_client = chromadb.Client()
## 创建集合
# collection = chroma_client.delete_collection(name="my_collection")
collection = chroma_client.create_collection(name="my_collection")


In [5]:
## 向集合中添加一些文本文档
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

/slurm/home/admin/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:33<00:00, 2.52MiB/s]


In [6]:
## 查询集合
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)


{'ids': [['id1', 'id2']], 'distances': [[1.0404009819030762, 1.2430799007415771]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


## 启动持久 Chroma 客户端

In [7]:
client = chromadb.PersistentClient(path="/slurm/home/admin/nlp/DL/tests/chroma_vextor")




In [8]:
client.heartbeat() # client是否连接


1721022626040168606

In [9]:
client.reset() # 清空数据，不可逆 ALLOW_RESET=TRUE` in your environment variables

ValueError: Resetting is not allowed by this configuration (to enable it, set `allow_reset` to `True` in your Settings() or include `ALLOW_RESET=TRUE` in your environment variables)

## 在客户端/服务器模式下运行Chroma

Chroma 还可配置为以客户端/服务器模式运行。在此模式下，Chroma 客户端连接到在单独进程中运行的 Chroma 服务器。

要启动 Chroma 服务器，请运行以下命令：

In [ ]:
chroma run --path /db_path

In [10]:
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

Chroma 还提供了异步 HTTP 客户端。其行为和方法签名与同步客户端相同，但所有会阻塞的方法现在都是异步的。要使用它，请改为调用AsyncHttpClient：

In [12]:
import asyncio
import chromadb
async def main():
    client = await chromadb.AsyncHttpClient()
    collection = await client.create_collection(name="my_collection")
    await collection.add(
        documents=["hello world"],
        ids=["id1"]
    )
asyncio.run(main())

## 使用 Python HTTP 客户端
如果您在客户端-服务器模式下运行 Chroma，则可能不需要完整的 Chroma 库。相反，您可以使用轻量级的仅客户端库。 在这种情况下，您可以安装该chromadb-client软件包。此软件包是服务器的轻量级 HTTP 客户端，依赖性最小。

`pip install chromadb-client`

In [ ]:
import chromadb
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)
# Or for async usage:
async def main():
    client = await chromadb.AsyncHttpClient(host='localhost', port=8000)

## 使用集合
### 创建、检查和删除集合#
Chroma 在 URL 中使用集合名称，因此命名时有一些限制：

- 名称的长度必须介于 3 到 63 个字符之间。
- 名称必须以小写字母或数字开头和结尾，中间可以包含点、破折号和下划线。
- 名称不得包含两个连续的点。
- 该名称不能是有效的 IP 地址。

Chroma 集合使用名称和可选的嵌入函数创建。如果您提供嵌入函数，则每次获取集合时都必须提供该函数。

In [ ]:
collection = client.create_collection(name="my_collection", embedding_function=emb_fn)
collection = client.get_collection(name="my_collection", embedding_function=emb_fn)


嵌入函数以文本为输入，并执行标记化和嵌入。如果没有提供嵌入函数，Chroma 将默认使用[句子转换器](https://www.sbert.net/index.html)。
> 您可以了解有关🧬[嵌入函数的更多信息](https://docs.trychroma.com/guides/embeddings)，以及如何创建自己的函数。

可以使用 按照名称检索现有集合`.get_collection`，并使用 删除集合`.delete_collection`。您还可以使用`.get_or_create_collection`获取集合（如果存在）或创建集合（如果不存在）。


In [ ]:
collection = client.get_collection(name="test") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
collection = client.get_or_create_collection(name="test") # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
client.delete_collection(name="my_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible


集合有一些实用的便捷方法。

In [ ]:
collection.peek() # returns a list of the first 10 items in the collection
collection.count() # returns the number of items in the collection
collection.modify(name="new_name") # Rename the coll

In [ ]:
## 距离函数
collection = client.create_collection(
    name="collection_name",
    metadata={"hnsw:space": "cosine"} # l2 is the default
)


有效选项为hnsw:space“l2”、“ip”或“cosine”。默认值为“l2”，即 L2 范数的平方。
| Distance | parameter |  |
| :--- | :--- | :--- |
| Squared L2 | 12 | $d=\sum\left(A_i-B_i\right)^2$ |
| Inner product | ip | $d=1.0-\sum\left(A_i \times B_i\right)$ |
| Cosine similarity | cosine | $d=1.0-\frac{\sum\left(A_i \times B_i\right)}{\sqrt{\sum\left(A_i^2\right)} \cdot \sqrt{\sum\left(B_i^2\right)}}$ |

### 向集合中添加数据




In [ ]:
collection.add(
    documents=["lorem ipsum...", "doc2", "doc3", ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]
)

如果提供的embeddings尺寸与集合的尺寸不一样，就会引发异常。

embeddings您还可以将文档存储在其他地方，只需向 Chroma 提供和列表即可metadata。您可以使用ids将嵌入与存储在其他地方的文档关联起来。

In [ ]:
collection.add(
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]
)


使用该方法，可以用多种方式查询色度集合`.query`。

您可以通过一组 进行查询`query_embeddings`。

In [ ]:
collection.query(
    query_embeddings=[[11.1, 12.1, 13.1],[1.1, 2.3, 3.2], ...],
    n_results=10,
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"}
)


查询将按顺序返回n_results与每个 最接近的匹配项。可以提供可选的过滤词典，以便根据与每个文档关联的 进行过滤。此外，还可以提供可选的过滤词典，以便根据文档的内容进行过滤。query_embedding where metadata where_document

如果提供的query_embeddings尺寸与集合的尺寸不一样，就会引发异常。

您还可以通过一组 进行查询query_texts。Chroma 将首先query_text使用集合的嵌入函数对每个 进行嵌入，然后使用生成的嵌入执行查询。

In [ ]:
collection.query(
    query_texts=["doc10", "thus spake zarathustra", ...],
    n_results=10,
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"}
)


id您还可以使用从集合中检索项目`.get`。


In [ ]:
collection.get(
	ids=["id1", "id2", "id3", ...],
	where={"style": "style1"}
)


.get还支持where和where_document过滤器。如果没有ids提供，它将返回集合中符合where和where_document过滤器的所有项目。

选择返回哪些数据
使用 get 或 query 时，您可以使用 include 参数指定要返回的数据 - 、、和 中的任意一个（embeddings对于query，）documents。默认情况下，Chroma 将返回，对于 query，返回结果中的 。默认情况下，为了提高性能，将排除 ，并且始终返回 。您可以通过将包含的字段名称数组传递给 query 或 get 方法的 includes 参数来指定要返回哪些数据。

In [ ]:

# Only get documents and ids
collection.get(
    include=["documents"]
)

collection.query(
    query_embeddings=[[11.1, 12.1, 13.1],[1.1, 2.3, 3.2], ...],
    include=["documents"]
)


### 使用 Where 过滤器
metadata Chroma 支持按和内容过滤查询document。where过滤器用于按 过滤metadata，where_document过滤器用于按document内容过滤。

按元数据过滤
为了过滤元数据，您必须where为查询提供过滤字典。该字典必须具有以下结构：

```python
{
    "metadata_field": {
        <Operator>: <Value>
    }
}

```



过滤元数据支持以下运算符：

- $eq- 等于（字符串，整数，浮点数）
- $ne- 不等于（字符串、整数、浮点数）
- $gt- 大于（int，float）
- $gte- 大于或等于（int，float）
- $lt- 小于（int，float）
- $lte- 小于或等于 (int, float)

使用 $eq 运算符相当于使用where过滤器。

```json
{
    "metadata_field": "search_string"
}

# is equivalent to

{
    "metadata_field": {
        "$eq": "search_string"
    }
}


```

#### 使用逻辑运算符

您还可以使用逻辑运算符`$and`和`$or`来组合多个过滤器。

运算符`$and`将返回与列表中所有过滤器匹配的结果。

```
{
    "$and": [
        {
            "metadata_field": {
                <Operator>: <Value>
            }
        },
        {
            "metadata_field": {
                <Operator>: <Value>
            }
        }
    ]
}

```

#### 使用包含运算符（$in和$nin）
支持以下包含运算符：

- $in- 值在预定义列表中（字符串、整数、浮点数、布尔值）
- $nin- 值不在预定义列表中（字符串、整数、浮点数、布尔值）
运算符$in将返回元数据属性属于所提供列表的一部分的结果：

```
{
  "metadata_field": {
    "$in": ["value1", "value2", "value3"]
  }
}

```


### 更新集合中的数据
可以使用 更新集合中项目的任何属性.update。


In [ ]:
collection.update(
    ids=["id1", "id2", "id3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    documents=["doc1", "doc2", "doc3", ...],
)


如果id在集合中未找到，则会记录错误并忽略更新。如果documents提供了但没有相应的embeddings，则将使用集合的嵌入函数重新计算嵌入。

如果提供的embeddings尺寸与集合的尺寸不一样，就会引发异常。

Chroma 还支持一项upsert操作，该操作可以更新现有项目，如果它们尚不存在则添加它们。

In [ ]:
collection.upsert(
    ids=["id1", "id2", "id3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    documents=["doc1", "doc2", "doc3", ...],
)


如果id集合中不存在，则将根据 创建相应的项目add。具有现有 的项目id将根据 进行更新update。

### 从集合中删除数据
idChroma 支持使用从集合中删除项目.delete。与每个项目相关的嵌入、文档和元数据都将被删除。⚠️ 当然，这是一个破坏性操作，无法撤消。

In [ ]:
collection.delete(
    ids=["id1", "id2", "id3",...],
	where={"chapter": "20"}
)
